# Call Center Analysis using Jupyter Notebook

This notebook demonstrates the entire data flow and tasks to analyze Customer Audio files using Microsoft Azure Cognitive services, Azure Blob Store and Azure SQL Data Warehouse. 



In [ ]:
import requests
from pprint import pprint
import urllib.request, json
import csv
import uuid
import operator
from difflib import SequenceMatcher
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
from time import time
import pyodbc
import os
import io
#import tables
import time
import azure
#from azure.storage.blob import BlockBlobService
import http.client, urllib.request, urllib.parse, urllib.error, base64
import logging
import sys
import json

In [ ]:
class Finding(object):
    cx_id = ""
    cx_feedback_theme = ""
    cx_intent = ""
    cx_keywords =""
    cx_channel_id =""
    cx_named_entity=""
    cx_transcript_offset=""
    cx_raw_transcript=""
    cx_transcript_confidence=""
    cx_sentiment=""
    
    # The class "constructor" - It's actually an initializer 
    def __init__(self, id, offset):
        self.cx_id = id
        self.cx_transcript_offset=offset

    def set_cx_raw_transcript(self, cx_raw_transcript):
        self.cx_raw_transcript=cx_raw_transcript

    def set_cx_transcript_confidence(self, cx_transcript_confidence):
        self.cx_transcript_confidence=cx_transcript_confidence

    def set_cx_sentiment(self, cx_transcript_offset):
        self.cx_transcript_offset=cx_transcript_offset

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,format="%(asctime)s %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p %Z")

## NLP using Azure Cognitive Service 

### Speech To Text

In [ ]:
serviceHostname = "https://eastus.api.cognitive.microsoft.com"
service_region = "eastus"
name = "Call Center Conversation Transcription"
description = "Call Center Conversation Sppech to Text Transcription Demo"
locale = "en-US"

# replace with the Azure Cognitive Speech Key
sub_key_speech = ""

#identifier for transption finding
stt_findings_id="123456789findings1"

### Replace with Call Center Recordings Samples
recordings_blob_uri_inbound_1 =""
recordings_blob_uri_inbound_2 = ""
recordings_blob_uri_outbound_1 = ""
recordings_blob_uri_outbound_2 = ""


In [ ]:
headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': sub_key_speech,
        }

body = {
        'contentUrls': [ recordings_blob_uri_inbound_1, 
                         recordings_blob_uri_inbound_2,
                         recordings_blob_uri_outbound_1,
                         recordings_blob_uri_outbound_2
        ],
        'properties': {
                      },
        'locale': locale,
        'displayName': name
}

In [ ]:
def transcribe():
    logging.info("Starting call center batch transcriptions...")
    segment_results = []
    
    try:
        conn = http.client.HTTPSConnection('eastus.api.cognitive.microsoft.com')
        conn.request("POST", "/speechtotext/v3.0/transcriptions", body=json.dumps(body), headers=headers)
        data = conn.getresponse().read().decode('utf-8')
        json_obj = json.loads(data)
        transcriptions_id = json_obj['links'].get('files').replace("/files", "").replace("https://eastus.api.cognitive.microsoft.com/speechtotext/v3.0/transcriptions/", "")
        
        # Log information about the created transcription. If you should ask for support, please
        # include this information.
        logging.info(f"Created new transcription with id '{transcriptions_id}' in region {service_region}")
        logging.info("Checking status .....")

        completed = False
        while not completed:
            # wait for 60 seconds before refreshing the transcription status
            time.sleep(60)
            
            conn.request("GET", f"/speechtotext/v3.0/transcriptions/{transcriptions_id}", body=json.dumps({}), headers=headers)
            data = conn.getresponse().read().decode('utf-8')
            #print(data)
            json_obj = json.loads(data)
            status = json_obj['status']
            logging.info(f"Transcriptions status: {status}")

            if status in ("Failed", "Succeeded"):
                completed = True

            if status == "Succeeded":
                
                conn = http.client.HTTPSConnection('eastus.api.cognitive.microsoft.com')
                conn.request("GET", f"/speechtotext/v3.0/transcriptions/{transcriptions_id}/files", body=json.dumps({}), headers=headers)
                response = conn.getresponse()
                data = response.read().decode('utf-8')
                json_obj = json.loads(data)

                for value in json_obj['values']:
                    if value.get('kind') == "Transcription":
                        results_url = value.get('links').get('contentUrl')
                        results = requests.get(results_url).content.decode('utf-8')
                        logging.info(f"Results for {results_url}:\n{results}")
                        segment_results.append(results)
               
            elif status == "Failed":
                logging.info("Transcription failed .......")
    
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return segment_results
        

In [ ]:
segment_results = transcribe()

In [ ]:
findings = []

for segment in segment_results:    
    data = json.loads(segment)
    
    for phrase in data['recognizedPhrases']:
        cx_finding=Finding(stt_findings_id, phrase.get('offsetInTicks'))
        cx_finding.cx_channel_id=phrase.get('channel')
            
        nBests = phrase.get('nBest')
        largest_confidence = nBests[0].get('confidence')
        
        for nBest in nBests:
            if nBest.get('confidence') > largest_confidence:
                largest_confidence = nBest.get('confidence')
            
        for nBest in nBests:
            if nBest.get('confidence') == largest_confidence:
                logging.info(f"Best transctiption record : {nBest}")
                cx_finding.cx_transcript_confidence=nBest.get('confidence')
                cx_finding.cx_raw_transcript=nBest.get('display')
                    
        findings.append(cx_finding)

Reteive the STT Results

In [ ]:
lines = []
dict_findings={}

for finding in findings:
    line = [ finding.cx_id, finding.cx_channel_id, finding.cx_feedback_theme, finding.cx_intent,  finding.cx_keywords, finding.cx_named_entity, finding.cx_transcript_offset, finding.cx_raw_transcript, finding.cx_transcript_confidence, finding.cx_sentiment  ]
    lines.append(line)
    dict_findings[str(finding.cx_transcript_offset)]=finding

for key, finding in dict_findings.items():  
    print(key, finding.cx_channel_id, finding.cx_transcript_offset, finding.cx_feedback_theme, finding.cx_intent, finding.cx_keywords, finding.cx_raw_transcript, finding.cx_transcript_confidence, finding.cx_sentiment)

In [ ]:
for key, finding in dict_findings.items():  
   finding.cx_keywords=""
   finding.cx_named_entity=""
    
for key, finding in dict_findings.items():  
    print(key, finding.cx_channel_id,  "|", finding.cx_named_entity, "|", finding.cx_transcript_offset,  "|", finding.cx_feedback_theme,  "|", finding.cx_intent, finding.cx_keywords,  "|", finding.cx_raw_transcript,  "|", finding.cx_transcript_confidence,  "|",  finding.cx_sentiment)


## Text Analytic

In [ ]:
# replace with Azure Text Analytics Service Key
sub_key_nlp = ""

### Sentiment Analysis

In [ ]:
headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': sub_key_nlp,
        }

# Request parameters   
params = urllib.parse.urlencode ({
    'showStats': 'false',
    'loggingOptOut': 'false',
    'opinionMining': 'true',
    'stringIndexType': 'TextElement_v8',
})

In [ ]:
def sentimentanalysis():
    logging.info("Starting Text Sentiment Analysis ......")
        
    try:
        conn = http.client.HTTPSConnection('languageSrv2.cognitiveservices.azure.com')
        conn.request("POST", "/text/analytics/v3.1/sentiment?%s" % params, json.dumps(stt_documents), headers)
    
        response = conn.getresponse().read().decode('utf-8')
        data = json.loads(response)
        sentiments_results = data['documents']
        # print(sentiments_results)
        for sentiment in sentiments_results:
            offset=str(sentiment["id"])
            confidence=sentiment["sentiment"]
            dict_findings.get(offset).cx_sentiment = confidence
            print(dict_findings.get(offset).cx_transcript_offset, dict_findings.get(offset).cx_raw_transcript, dict_findings.get(offset).cx_transcript_confidence, dict_findings.get(offset).cx_sentiment)
        
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    

In [ ]:
# Initialize Counter
stt_documents={}
documents_list=[]
id=1

for finding in findings:
    value_dict = {
                'id': str(finding.cx_transcript_offset),
                'language': 'en',
                'text': finding.cx_raw_transcript
              }
    
    documents_list.append(value_dict)
    id += 1
    
    if id > 10 :
        stt_documents["documents"] = documents_list
        ## Reset Counter
        sentimentanalysis()
        stt_documents={}
        documents_list=[]
        id = 1
    
stt_documents["documents"] = documents_list
sentimentanalysis()
#pprint(stt_documents)

### Key Phrases Extraction

In [ ]:
dict_feedback_theme={}
dict_feedback_theme["Advertisement"] = ['information', 'question', 'account', 'program requirements', 'policies']
dict_feedback_theme["Communication"] = ['shipping','USPS', 'customer service', 'exchange','return']

dict_cx_intent={}
                
dict_cx_intent["Shipping"] = ['shipping', 'USPS', 'place order','product']
dict_cx_intent["Verification"] = ['phone', 'email', 'questions', 'policies']
dict_cx_intent["Order"] = ['order Number', 'shipping ETA', 'arrive','status']
dict_cx_intent["Product"] = ['availability','product']
dict_cx_intent["Return"] = ['return label', 'black box', 'price error', 'wrong price']
dict_cx_intent["Documentation"] = ['Invoice Copy','Packing List']


In [ ]:
def analyzeintent(key_phrases_results):

    for key_phrases in key_phrases_results:
        offset=str(key_phrases["id"])
        key_phrases_list=key_phrases["keyPhrases"]
        
        intent_hit = []
        keywords_hit = []
        theme_hit = []
            
            
        if len(key_phrases_list) != 0: 
            #print(key_phrases_list) 
            intent_hit = []
            keywords_hit = []
            theme_hit = []
            
            for key_phrase in key_phrases_list:
                # loop through dict_cx_intent
                for key, keywords in dict_cx_intent.items():
                    # loop through keywords list
                    for keyword in keywords:
                        matcher = SequenceMatcher(None, key_phrase.lower(), keyword.lower())
                        # setup bar value as 85% similarity
                        if (matcher.ratio() > 0.85) :
                            #print ("comparing %s with %s: similarity is %s" % (key_phrase, keyword, matcher.ratio()))
                        
                            if key not in intent_hit :
                               intent_hit.append(key)
                        
                            # loop through theme list
                            for theme, values in dict_feedback_theme.items():
                                for value in values:
                                    #print ("comparing %s with %s: %s" % (value, key, key == value)) 
                            
                                    if (value == key) : 
                                        if theme not in theme_hit :
                                            theme_hit.append(theme)
                        
                            if key_phrase not in keywords_hit :
                               keywords_hit.append(key_phrase)
                
                if (dict_findings.get(offset).cx_keywords.strip().find(key_phrase) == -1):
                    # print("adding ..." + key_phrase)
                    dict_findings.get(offset).cx_keywords += key_phrase + ","       
                        
        for intent in intent_hit:        
            if not dict_findings.get(offset).cx_intent.strip():
                    dict_findings.get(offset).cx_intent += intent
            else :
                dict_findings.get(offset).cx_intent += "," + intent
   
        for theme in theme_hit:        
            if not dict_findings.get(offset).cx_feedback_theme.strip():
                dict_findings.get(offset).cx_feedback_theme += theme
            else :
                dict_findings.get(offset).cx_feedback_theme += "," + theme
    
        # for kw in keywords_hit:
        #    if not dict_findings.get(offset).cx_keywords.strip():
        #        dict_findings.get(offset).cx_keywords += kw       
        #    else :
        #        dict_findings.get(offset).cx_keywords += "," + kw 
        
        
        print(dict_findings.get(offset).cx_transcript_offset, "|", dict_findings.get(offset).cx_keywords, "|", dict_findings.get(offset).cx_feedback_theme, "|", dict_findings.get(offset).cx_intent, "|", dict_findings.get(offset).cx_raw_transcript, "|", dict_findings.get(offset).cx_transcript_confidence, "|", dict_findings.get(offset).cx_sentiment)
    

In [ ]:
def keyphraseextraction():
    logging.info("Starting Key Phrase Extraction ......")
        
    try:
        conn = http.client.HTTPSConnection('languageSrv2.cognitiveservices.azure.com')
        conn.request("POST", "/text/analytics/v3.1/keyPhrases?%s" % params, json.dumps(stt_documents), headers)
    
        response = conn.getresponse().read().decode('utf-8')
        data = json.loads(response)
        key_phrases_results = data['documents']
        #print(key_phrases_results)
        analyzeintent(key_phrases_results)
        
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    

In [ ]:
# Initialize Counter
stt_documents={}
documents_list=[]
id=1

for finding in findings:
    value_dict = {
                'id': str(finding.cx_transcript_offset),
                'language': 'en',
                'text': finding.cx_raw_transcript
              }
    
    documents_list.append(value_dict)
    id += 1
    
    if id > 10 :
        stt_documents["documents"] = documents_list
        ## Reset Counter
        keyphraseextraction()
        stt_documents={}
        documents_list=[]
        id = 1
    
stt_documents["documents"] = documents_list
keyphraseextraction()

### Named Entity Recognition

In [ ]:
def extractner(ner_results):

    for ner in ner_results:
        offset=str(ner["id"])
        entity_list=ner["entities"]

        for entity in entity_list: 
            # print(entity.get('category'))
            category = entity.get('category')
            
            if (dict_findings.get(offset).cx_named_entity.strip().find(category) == -1):
                # print("adding ..." + category)
                dict_findings.get(offset).cx_named_entity += category + ","    
    
        print(dict_findings.get(offset).cx_transcript_offset, "|", dict_findings.get(offset).cx_named_entity, "|", dict_findings.get(offset).cx_feedback_theme, "|", dict_findings.get(offset).cx_intent, "|", dict_findings.get(offset).cx_keywords, "|", dict_findings.get(offset).cx_raw_transcript, "|", dict_findings.get(offset).cx_transcript_confidence, "|", dict_findings.get(offset).cx_sentiment)
    

In [ ]:
def nameentityrecognition():
    logging.info("Starting Named Entity Recognition ......")
        
    try:
        conn = http.client.HTTPSConnection('languageSrv2.cognitiveservices.azure.com')
        conn.request("POST", "/text/analytics/v3.1/entities/recognition/general?%s" % params, json.dumps(stt_documents), headers)
    
        response = conn.getresponse().read().decode('utf-8')
        data = json.loads(response)
        ner_results = data['documents']
        #print(ner_results)
        extractner(ner_results)
        
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    

In [ ]:
# Initialize Counter
stt_documents={}
documents_list=[]
id=1

for finding in findings:
    value_dict = {
                'id': str(finding.cx_transcript_offset),
                'language': 'en',
                'text': finding.cx_raw_transcript
              }
    
    documents_list.append(value_dict)
    id += 1
    
    if id > 5 :
        stt_documents["documents"] = documents_list
        ## Reset Counter
        nameentityrecognition()
        stt_documents={}
        documents_list=[]
        id = 1
    
stt_documents["documents"] = documents_list
nameentityrecognition()

### Output the Key Fidning Results to Local CSV file

In [ ]:
csv_lines = []

for key, finding in dict_findings.items():  
    csv_line = [ finding.cx_id, finding.cx_feedback_theme, finding.cx_intent,  finding.cx_keywords, finding.cx_named_entity, finding.cx_transcript_offset, finding.cx_raw_transcript, finding.cx_transcript_confidence, finding.cx_sentiment  ]
    csv_lines.append(csv_line)
    

In [ ]:
stt_findings_csv="123456789findings1.csv"

header=['cx_id','cx_feedback_theme', 'cx_intent','cx_keywords','cx_named_entity','cx_transcript_offset','cx_raw_transcript','cx_transcript_confidence','cx_sentiment']

with open(stt_findings_csv, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(header)
    
    writer.writerows(csv_lines)

### Output the Key Fidning Results to Azure Blob Store

In [ ]:

# replace with Azure Blob Storage name and folder
blob_store_account_name = ""
audio_container_name = ''
finding_container_name = ''
filefolder=""
## Replace with Azure Blob Account Key
blob_storage_key = ""

block_blob_service = BlockBlobService(account_name=blob_store_account_name, account_key=blob_storage_key)

In [ ]:
# replace with findings file name
stt_findings_csv=""

header=['cx_id','cx_feedback_theme', 'cx_intent','cx_keywords','cx_named_entity','cx_transcript_offset','cx_raw_transcript','cx_transcript_confidence','cx_sentiment']

with open(stt_findings_csv, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(header)
    
    writer.writerows(csv_lines)

In [ ]:
full_path_to_file = os.path.join(os.getcwd(), stt_findings_csv)
print(full_path_to_file)

block_blob_service.create_blob_from_path(finding_container_name, stt_findings_csv, full_path_to_file)

Write a csv file via pandas

In [ ]:
output = io.StringIO()
df = pd.DataFrame (csv_lines , columns = header)
#print(df)
output = df.to_csv (index_label="idx", encoding = "utf-8")
print(output)

block_blob_service.create_blob_from_text(finding_container_name, stt_findings_csv, output)

### Loading STT analytic finding dataset into a Pandas data frame, visualizing and exploring the data

In [ ]:
# replace with Azure SQL DB Credential 
server_name = ''
database_name   = ''
userid        = ''
password      = ''
db_driver     = 'ODBC Driver 13 for SQL Server'

#Driver={ODBC Driver 13 for SQL Server};Server=tcp:jnjsstdw.database.windows.net,1433;Database=jnjsttfinding;Uid=jnjsstadmin@jnjsstdw;Pwd={your_password_here};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;

In [ ]:
# Construct the SQL DW Connection string
driver = 'DRIVER={' + db_driver + '}'
server = 'SERVER=' + server_name 
database = 'DATABASE=' + database_name
uid = 'UID=' + userid 
pwd = 'PWD=' + password
connection_string = ';'.join([driver,server,database,uid,pwd, 'Encrypt=yes;TrustServerCertificate=no'])
print(connection_string)

In [ ]:
# Connect to the Database. Autocommit needs to turned on for DDL statements
conn = pyodbc.connect(connection_string)
conn.autocommit=True
cursor = conn.cursor()

Report number of rows and columns in table jnjcx.sstfindings

In [ ]:
nrows = pd.read_sql('''SELECT SUM(rows) FROM sys.partitions WHERE object_id = OBJECT_ID('jnjcx.sst_findings')''', conn)
print ('Total number of rows = %d' % nrows.iloc[0,0])

ncols = pd.read_sql('''SELECT count(*) FROM information_schema.columns WHERE table_name = ('sst_findings') AND 
table_schema = ('jnjcx')''', conn)
print ('Total number of columns = %d' % ncols.iloc[0,0])

Loading the dataset into Pandas DF 

In [ ]:
df1 = pd.read_sql('''select top 10000 * from jnjcx.sst_findings t ''', conn)

In [ ]:
df1['cx_transcript_confidence'].describe()

In [ ]:
df1.boxplot(column='cx_transcript_confidence',return_type='dict')

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
df1['cx_transcript_confidence'].plot(ax=ax1,kind='kde', style='b-')
df1['cx_transcript_confidence'].hist(ax=ax2, bins=100, color='k')

In [ ]:
conf_dist_bins = [0, 0.2, .4, 0.6, 0.8, 1]
df1['cx_transcript_confidence']
conf_dist_bin_id = pd.cut(df1['cx_transcript_confidence'], conf_dist_bins)
conf_dist_bin_id

In [ ]:
pd.Series(conf_dist_bin_id).value_counts()

In [ ]:
pd.Series(conf_dist_bin_id).value_counts().plot(kind='bar')

In [ ]:
pd.Series(conf_dist_bin_id).value_counts().plot(kind='line')

In [ ]:
df1 = pd.read_sql('''select top 10000 * from jnjcx.sst_findings t where t.cx_id='736021005637009' ''', conn)

In [ ]:
plt.scatter(df1['cx_transcript_offset'], df1['cx_transcript_confidence'])

Load Data for a specific audio file

In [ ]:
query = '''
        SELECT *
        FROM jnjcx.sst_findings t
        where t.cx_id='736021005637009'
        order by t.cx_transcript_offset
        '''

pd.read_sql(query, conn)